# Transcript data feature engineering

### Loading Packages

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

2022-10-23 18:13:13.685015: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 18:13:13.815642: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-23 18:13:14.412310: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib.real:/opt/conda/lib/python3.8/site-packages/torch/lib:/opt/conda/lib/python3.8/site-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64


['/device:CPU:0', '/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7']


2022-10-23 18:13:15.338011: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 18:13:19.187860: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 34652 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2022-10-23 18:13:19.189752: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 30828 MB memory:  -> device: 1, name: A100-SXM4-40GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
2022-10-23 18:13:19.191493: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:2 with 37221 MB memory:  -> device: 2, name: A100-SXM4-40GB, pci bus id: 0000:47:00.0, compute capab

In [4]:
#import os
#os.environ['CUDA_VISIBLE_DEVICES']

In [5]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [6]:
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [7]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [8]:
import pandas as pd
import numpy as np
from transformers import pipeline
from tqdm import tqdm
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import time
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading datasets

In [9]:
%cd ..

/workspace/asurion_f22_muby


In [10]:
transcript_train = pd.read_csv("fe_data_v2/transcript_data/transcript_train_data.csv")

In [68]:
transcript_train["encrypted_collated_transcription"] = transcript_train["encrypted_collated_transcription"].astype(str)

/tmp/ipykernel_23934/455747962.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcript_train["encrypted_collated_transcription"] = transcript_train["encrypted_collated_transcription"].astype(str)
/tmp/ipykernel_23934/455747962.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcript_test["encrypted_collated_transcription"] = transcript_test["encrypted_collated_transcription"].astype(str)


In [11]:
transcript_train.head()

,id,sales_offer_date,accepted_flg,encrypted_collated_transcription,month,length,length_3rd,first_3rd,second_3rd,third_3rd
0,e5e4ee3c3c0e9ac72561334e6eb23cc233b1bcd0a0063d...,2022-03-26,False,ok Thank for calling [CLIENT] Tech Hello? Hi. ...,march,7054,2351,ok Thank for calling [CLIENT] Tech Hello? Hi. ...,et you know what I could be able to see here. ...,"see is that the, uh, places is still under th..."
1,ae6128d8d240d28edbcb7eb27256780cf662a2be1aec8c...,2022-03-02,False,"on on Oh, My name is [NAME]. May I please have...",march,7467,2489,"on on Oh, My name is [NAME]. May I please have...","y got all of that Well, their account for hack...","I they're setting up their Myverizon account, ..."
2,4899f9acdb70b0246b2d0ac2871dbcea8267e57b060154...,2022-03-12,False,Hello. We for calling [CLIENT] Tech Coach. My ...,march,3953,1317,Hello. We for calling [CLIENT] Tech Coach. My ...,"know, do I hit. get through the put me throug...","t to Oh, I can I just another phone sir maybe,..."
3,0589751e11caac0eac815faf84f1a366d70b86b78324a2...,2022-03-20,False,Hello. Thank you so much for contacting [CLIEN...,march,6998,2332,Hello. Thank you so much for contacting [CLIEN...,et it many times. Good there is what we call a...,"ing issues being able to send text messages, t..."
4,6c4790cb02e4f556d73436f79268d5c99de50a9acec40e...,2022-03-15,False,Thank you very much Hello? can I can you hear ...,march,7550,2516,Thank you very much Hello? can I can you hear ...,something happened with the features. I had to...,hold. uh ok. rode um. Man. um. woo Well? thank...


#### Creating Summarization Function

In [13]:
from transformers import pipeline

In [14]:
pega = pipeline("summarization", model="google/pegasus-cnn_dailymail")

In [15]:
transcript_train.iloc[0,3][:3500]

"ok Thank for calling [CLIENT] Tech Hello? Hi. It's Christie Ferguson. Speaking. May I have your first and last name, please? Hi there, Christie. What's the phone number calling about? I'm calling about mobile, um, data use constantly anymore. Ok. Well, sorry here they have the issues with regards your Mobile data about, Christie. What's the phone number you're calling about? Seven [NUMBER] [NUMBER] [NUMBER] [NUMBER] [NUMBER] [NUMBER] [NUMBER] [NUMBER] seven Thank you so much. Ok. Let me just pull up your account here, Chris. so Could be able to run a network test and then uh later on let's make sure that all of the security pictures activated onto your account. So if could be able to maximize the uh, functionality see every phone. About the way Christie, uh, you mentioned and that you're having troubles when it comes to your cellular data Can you tell me since when it started happening? been going on and off for probably a couple weeks, but if it's not Probably. know, one thing it's a

In [16]:
pipe_out_pega = pega(transcript_train.iloc[2,3][:3000])
summary = pipe_out_pega[0]["summary_text"].replace(" .<n>", ". ")
summary

'When calling [CLIENT] Tech Coach, the operator will try to connect you to a different number. If this happens, the operator will try to reset your phone number. When this happens, the operator will try to connect you to a different number. If this happens again, the operator will try to reset your phone number .'

In [17]:
pipe_out_pega = pega(transcript_train.iloc[16,3][:3000])
summary = pipe_out_pega[0]["summary_text"].replace(" .<n>", ". ")
summary

"Learn how to reset your phone so it doesn't freeze when you try to use it. Turn on your TV so it doesn't freeze when you try to use it. Use the weekly Newsquiz to test your knowledge of stories you saw on CNN.com ."

In [18]:
#for i in range(len(transcript_train)):
#    pipe_out_pega = pega(transcript_train.iloc[i,3][:3500])
#    transcript_train.loc[i,"summary"] = pipe_out_pega[0]["summary_text"].replace(" .<n>", ". ")

The summarization function has a lot of limitations with regards to usage. Some of its issues are highlighted below:
- Limitation of input text length: One observance made was that the summarization models could not process text that were beyond a certain number of words/characters/tokens. Hence, the text going into summarization models had to be truncated.
- Wrong Output Provision: Due to the fact that the different summarization models have been pre-trained on certain data, they might end up providing output that has nothing to do with the context of the conversation between the client and the Asurion rep. This might stem from the fact that some conversations were literally pointless and had no direction. With the summarization models being coerced into saying something, they are likely to bring up some random summary they had to do in the past

### Sentiment Analysis

#### 1st approach

In [ ]:
#max_memory_mapping = {0: "40GB"}
#model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli") # device_map = "auto",load_in_8bit=True, max_memory=max_memory_mapping)
#tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")
#classifier = pipeline("zero-shot-classification",
#                      model=model, tokenizer = tokenizer, device = 0,framework = "pt")

In [19]:
classifier = pipeline("zero-shot-classification", model = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",device = 0,framework = "pt", num_workers = 1)

/opt/conda/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [20]:
classifier(transcript_train.iloc[0,3], ["anger", "happiness", "neutral","confusion","sadness"], multi_label = True)

/opt/conda/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:745: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(
/opt/conda/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:829: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  score += c2p_att / torch.tensor(scale, dtype=c2p_att.dtype)
/opt/conda/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:852: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather 

{'sequence': "ok Thank for calling [CLIENT] Tech Hello? Hi. It's Christie Ferguson. Speaking. May I have your first and last name, please? Hi there, Christie. What's the phone number calling about? I'm calling about mobile, um, data use constantly anymore. Ok. Well, sorry here they have the issues with regards your Mobile data about, Christie. What's the phone number you're calling about? Seven [NUMBER] [NUMBER] [NUMBER] [NUMBER] [NUMBER] [NUMBER] [NUMBER] [NUMBER] seven Thank you so much. Ok. Let me just pull up your account here, Chris. so Could be able to run a network test and then uh later on let's make sure that all of the security pictures activated onto your account. So if could be able to maximize the uh, functionality see every phone. About the way Christie, uh, you mentioned and that you're having troubles when it comes to your cellular data Can you tell me since when it started happening? been going on and off for probably a couple weeks, but if it's not Probably. know, one

In [21]:
#import time
#import torch
#from torch.multiprocessing import Pool
#torch.multiprocessing.set_start_method('spawn', force=True)

In [22]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2,4,5,6,7"

In [23]:
for i in tqdm(range((100))):
    result = classifier(transcript_train.iloc[i,3], ['negative', 'neutral', 'positive'])
    labels = result["labels"]
    scores = result["scores"]
    for j in range(len(labels)):
        transcript_train.loc[i,labels[j]] = scores[j]

  9%|███████▍                                                                           | 9/100 [00:01<00:10,  8.49it/s]/opt/conda/lib/python3.8/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.47it/s]


In [24]:
transcript_train.head(100)

,id,sales_offer_date,accepted_flg,encrypted_collated_transcription,month,length,length_3rd,first_3rd,second_3rd,third_3rd,negative,neutral,positive
0,e5e4ee3c3c0e9ac72561334e6eb23cc233b1bcd0a0063d...,2022-03-26,False,ok Thank for calling [CLIENT] Tech Hello? Hi. ...,march,7054,2351,ok Thank for calling [CLIENT] Tech Hello? Hi. ...,et you know what I could be able to see here. ...,"see is that the, uh, places is still under th...",0.700241,0.215107,0.084652
1,ae6128d8d240d28edbcb7eb27256780cf662a2be1aec8c...,2022-03-02,False,"on on Oh, My name is [NAME]. May I please have...",march,7467,2489,"on on Oh, My name is [NAME]. May I please have...","y got all of that Well, their account for hack...","I they're setting up their Myverizon account, ...",0.376522,0.284553,0.338926
2,4899f9acdb70b0246b2d0ac2871dbcea8267e57b060154...,2022-03-12,False,Hello. We for calling [CLIENT] Tech Coach. My ...,march,3953,1317,Hello. We for calling [CLIENT] Tech Coach. My ...,"know, do I hit. get through the put me throug...","t to Oh, I can I just another phone sir maybe,...",0.497945,0.367543,0.134512
3,0589751e11caac0eac815faf84f1a366d70b86b78324a2...,2022-03-20,False,Hello. Thank you so much for contacting [CLIEN...,march,6998,2332,Hello. Thank you so much for contacting [CLIEN...,et it many times. Good there is what we call a...,"ing issues being able to send text messages, t...",0.294379,0.249605,0.456016
4,6c4790cb02e4f556d73436f79268d5c99de50a9acec40e...,2022-03-15,False,Thank you very much Hello? can I can you hear ...,march,7550,2516,Thank you very much Hello? can I can you hear ...,something happened with the features. I had to...,hold. uh ok. rode um. Man. um. woo Well? thank...,0.340227,0.422270,0.237504
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,62e8b9b20b5d50cd13e5d6c3ee8deb2f8dc219c7bffeaf...,2022-03-03,False,Hi. Thank you for calling [CLIENT] Hi. Thank y...,march,33338,11112,Hi. Thank you for calling [CLIENT] Hi. Thank y...,messenger. Ok. My files. That's like the folde...,"s just in case, you know, you damaged through ...",0.420673,0.223538,0.355789
96,8702e4863398b306ad47570a8a012bb1925f94e922f051...,2022-03-01,False,Hello. Thank you for calling [CLIENT] Tech Coa...,march,7537,2512,Hello. Thank you for calling [CLIENT] Tech Coa...,nly thing I'm having problems about what do I ...,e you can also file a claim. So can you imagin...,0.051676,0.137461,0.810864
97,e683dc35c156bb2c1d05bb2541ac1d6d1dd9dca259ac5d...,2022-03-20,False,[NUMBER] Alright. Thank you for calling [CLIEN...,march,8034,2678,[NUMBER] Alright. Thank you for calling [CLIEN...,should account. slash find. Ok. what we went ...,tried to do it a couple of times. This only m...,0.271412,0.304093,0.424494
98,982893bda8254e71d6d8a82194f1bdd7efbe4833a7ad81...,2022-03-27,False,"Hello. Thank you for calling E. Hello? Coach, ...",march,36437,12145,"Hello. Thank you for calling E. Hello? Coach, ...",not not this Gp. Not the Wifi. Alright. You're...,doing it too long. I'm not what you have to d...,0.155648,0.288768,0.555584


In [25]:
np.arange(len(transcript_train), 8)

array([], dtype=int64)

In [26]:
len(transcript_train)/8

130553.875

In [27]:
130554*np.arange(8)

array([     0, 130554, 261108, 391662, 522216, 652770, 783324, 913878])

#### 2nd approach

In [28]:
max_memory_mapping = {0: "8GB", 1: "8GB", 2: "8GB", 3: "8GB", 4: "8GB", 5: "8GB", 6: "8GB", 7: "8GB"}
model = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli") # device_map = "auto",load_in_8bit=True, max_memory=max_memory_mapping)
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
classifier = pipeline("zero-shot-classification",
                      model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device = 3,framework = "pt", num_workers = 1)

/opt/conda/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [32]:
classifier = pipeline("zero-shot-classification", model = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",device = 0,framework = "pt", num_workers = 1)

In [33]:
first_3rd = transcript_train.loc[1000,"first_3rd"]
second_3rd = transcript_train.loc[1000,"second_3rd"]
third_3rd = transcript_train.loc[1000,"third_3rd"]
result1 = classifier(first_3rd, ["anger", "happiness", "neutral","sadness"])
print(result1["labels"][0], result1["scores"][0])
result2 = classifier(second_3rd, ["anger", "happiness", "neutral","sadness"])
print(result2["labels"][0], result2["scores"][0])
result3 = classifier(third_3rd, ["anger", "happiness", "neutral","sadness"])
print(result3["labels"][0], result3["scores"][0])

/opt/conda/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:745: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(
/opt/conda/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:829: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  score += c2p_att / torch.tensor(scale, dtype=c2p_att.dtype)
/opt/conda/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:852: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather 

sadness 0.3752005696296692
happiness 0.37023481726646423
neutral 0.4168095588684082


In [34]:
from tqdm import tqdm_notebook

In [39]:
for i in tqdm(range((100))):
    conversation = transcript_train.loc[i,"first_3rd"]
    result_1st = classifier(conversation, ["anger", "happiness", "neutral","sadness"])
    transcript_train.loc[i, "emotion_1st"] = result_1st["labels"][0]

  0%|                                                                                           | 0/100 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:745: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(
/opt/conda/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:829: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sou

In [40]:
transcript_train["emotion_1st"].head(100).value_counts()

neutral      47
sadness      29
happiness    13
anger        11
Name: emotion_1st, dtype: int64